## Installation

### Please Read This First

Delivering LSP features to your JupyterLab **requires** three pieces:

- `jupyter_lsp`
  - runs in your `notebook` web application on your server
  - to run, you need:
    - `python >=3.5`
    - `notebook >=4.3`
- `jupyterlab-lsp`
  - runs in your browser
  - to install it, you need:
    - `nodejs >8`
    - `jupyterlab >=1.1,<2`
- one or more language servers
  - run on your server
  - some can be automatically [detected](./Language%20Servers.ipynb) if
    installed
  - others also need to be [configured](./Configuring.ipynb#language_servers)

### Fast Paths

Here are two approches based on Jupyter documentation. If these do not meet your
needs, try [The Harder Way](#The-Harder-Way).

#### conda (minimal python)

```bash
conda create -c conda-forge -n lsp 'python >=3.7,<3.8' 'jupyterlab=1.2' 'nodejs=11' python-language-server
# Also consider: r-languageserver [*]
source activate lsp
pip install jupyter_lsp=0.7 --no-deps
jupyter lab build
jupyter lab
```

#### docker (data science)

This approach is based roughly on the
[Jupyter docker-stacks documentation](https://github.com/jupyter/docker-stacks/tree/master/examples/docker-compose/notebook).

##### `Dockerfile`

```dockerfile
FROM jupyter/datascience-notebook:18ef2702c6a2

RUN conda install --quiet --yes --freeze-installed \
        'jupyterlab=1.2' \
        'nodejs=11' \
        'python-language-server' \
        'r-languageserver' \
    && pip install \
        'jupyter_lsp=0.7' \
    && conda clean --all -f -y \
    && fix-permissions "$CONDA_DIR" \
    && fix-permissions "/home/$NB_USER" \
    && jupyter lab build \
    && jlpm cache clean \
    && rm -rf "$CONDA_DIR/share/jupyter/lab/{staging}"
```

##### `docker-compose.yml`

```yaml
version: '2'

services:
  notebook:
    build: .
    image: jupyterlab-lsp-notebook
    container_name: ${NAME}
    volumes:
      - 'work:/home/jovyan/work'
    ports:
      - '${PORT}:8888'
    environment:
      USE_HTTPS: 'yes'
      PASSWORD: ${PASSWORD}

volumes:
  work:
    external:
      name: ${WORK_VOLUME}
```

##### Build and Start

```bash
PORT=8899 NAME=lsp PASSWORD=change-me-silly WORK_VOLUME=. docker-compose up
```

### The Harder Way

#### Get A Working JupyterLab environment

Refer to the official
[JupyterLab Installation Documentation](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)
for your installation approach.

| pip            | conda            | pipenv | poetry | `*` |
| -------------- | ---------------- | ------ | ------ | --- |
| [lab][lab-pip] | [lab][lab-conda] | `*`    | `*`    | `*` |

> `*` PRs welcome!

Verify your lab works:

```bash
jupyter lab --version
jupyter lab
```

[lab-conda]:
  https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html#conda
[lab-pip]:
  https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html#pip

#### Get a Working NodeJS

The [JupyterLab Development Environment Documentation][nodejs-conda] shows some
approaches for getting NodeJS.

| conda                  | `*` |
| ---------------------- | --- |
| [nodejs][nodejs-conda] | `*` |

[nodejs-conda]:
  https://jupyterlab.readthedocs.io/en/stable/developer/extension_tutorial.html#set-up-a-development-environment

Verify your node works and is findable from python.

```bash
jlpm versions
```

#### Install Jupyter[Lab] LSP

```bash
pip install jupyter_lsp
jupyter labextension install @krassowitz/jupyterlab-lsp
```

### Next Step: Language Servers

Now that you have `jupyterlab-lsp`, `jupyter_lsp` and all of their dependencies,
you'll need some language servers. See:

- [Language Servers](./Language%20Servers.ipynb) that will be found
  automatically once installed
- [configuring](./Configuring.ipynb) `jupyter_lsp` for more control over which
  servers to load